In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %export PYTHONPATH='./'
import os
os.environ['PYTHONPATH']='./'
os.environ['OMP_NUM_THREADS']='1'
os.environ['PYTHONWARNINGS']='ignore'

In [28]:
!rm -r data

In [29]:
img_dir = '/data/DMS_Drowsiness/all_video_symlink/56fb756686fbf9796f2cff7af16684ac/croped_faces/'
real_ds = 'data/nguyenvandong_lmdb'

In [31]:
!python prepare_data.py --path {img_dir} --out {real_ds} --pad border --size 256 --pattern "*.jpg"

Make dataset of image sizes: 256
Found 53925 files
Example file being loaded: /data/DMS_Drowsiness/all_video_symlink/56fb756686fbf9796f2cff7af16684ac/croped_faces/000001.jpg
100%|███████████████████████████████████| 53925/53925 [00:33<00:00, 1618.77it/s]
Final dataset size: 53925


In [32]:
!torchrun --nproc_per_node=8 applications/flow_scores.py \
--ckpt 'ir_face' --real_data_path {real_ds} --real_size 256 --no_flip_inference

100%|█████████████████████████████████████████| 135/135 [00:48<00:00,  2.76it/s]
num_scores = 53925
Flow scores saved at data/nguyenvandong_lmdb/flow_scores.pt


In [26]:
# out_dir = f'{real_ds}_align'

In [33]:
! torchrun --nproc_per_node=1 applications/congeal_dataset.py \
--ckpt ir_face --real_data_path {real_ds} \
--out {out_dir} --real_size 256 --flow_scores {real_ds}/flow_scores.pt \
--fraction_retained 0.05 --output_resolution 256  \
--min_effective_resolution 50

2022-07-10 10:12:12.909 | INFO     | __main__:apply_congealing:30 - (rank=0) Using prefix a
  0%|                                        | 1/2697 [00:01<1:12:46,  1.62s/it]2022-07-10 10:12:14.625 | INFO     | __main__:apply_congealing:70 - Ignore becuase too_low_res=False, oob.item()=True
2022-07-10 10:12:14.719 | INFO     | __main__:apply_congealing:70 - Ignore becuase too_low_res=False, oob.item()=True
  0%|                                          | 3/2697 [00:01<21:58,  2.04it/s]2022-07-10 10:12:14.814 | INFO     | __main__:apply_congealing:70 - Ignore becuase too_low_res=False, oob.item()=True
2022-07-10 10:12:14.911 | INFO     | __main__:apply_congealing:70 - Ignore becuase too_low_res=False, oob.item()=True
  0%|                                          | 6/2697 [00:02<10:38,  4.22it/s]2022-07-10 10:12:15.110 | INFO     | __main__:apply_congealing:70 - Ignore becuase too_low_res=False, oob.item()=True
2022-07-10 10:12:15.205 | INFO     | __main__:apply_congealing:70 - Ignore bec